# A Dummy Model

## Preamble

In [7]:
import pandas

In [8]:
import matplotlib.pyplot as plt
plt.style.use('dark_background')

## Data Preparation

In [9]:
movies = catalog.load("movies")

2021-03-16 10:22:04,663 - kedro.io.data_catalog - INFO - Loading data from `movies` (ParquetDataSet)...


In [10]:
#movies = movies.set_index("id")

In [11]:
movies["genre_names"] = movies["genres"].apply(lambda ls: [d["name"] for d in ls])

In [12]:
movies = movies[movies["genre_names"].map(lambda d: len(d)) > 0] # drop  movies without genres

In [13]:
genre_labels = pandas.get_dummies(
    movies["genre_names"].apply(pandas.Series).stack(dropna=False),
    prefix="Genre"
).sum(level=0)

In [14]:
posters = movies[["original_title", "poster_path"]]

In [15]:
genre_labels

,Genre_Action,Genre_Adventure,Genre_Animation,Genre_Comedy,Genre_Crime,Genre_Documentary,Genre_Drama,Genre_Family,Genre_Fantasy,Genre_History,Genre_Horror,Genre_Music,Genre_Mystery,Genre_Romance,Genre_Science Fiction,Genre_TV Movie,Genre_Thriller,Genre_War,Genre_Western
id,,,,,,,,,,,,,,,,,,,
2,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
8,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
19996,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
19997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [16]:
movies[~movies.index.isin(genre_labels.index)]

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,...,tagline,title,video,vote_average,vote_count,belongs_to_collection.id,belongs_to_collection.name,belongs_to_collection.poster_path,belongs_to_collection.backdrop_path,genre_names
id,,,,,,,,,,,,,,,,,,,,,


## Metrics

In [17]:
from sklearn.metrics import f1_score

In [18]:
from sklearn.model_selection import cross_val_score

In [19]:
from sklearn.metrics import make_scorer

## Dummy Classifier

In [20]:
from sklearn.dummy import DummyClassifier

In [21]:
classifier = DummyClassifier(strategy="stratified")

In [22]:
classifier.fit(
    posters,
    genre_labels
)

DummyClassifier(strategy='stratified')

In [23]:
Y_pred = pandas.DataFrame(
    classifier.predict(posters),
    columns=genre_labels.columns
)
Y_pred

,Genre_Action,Genre_Adventure,Genre_Animation,Genre_Comedy,Genre_Crime,Genre_Documentary,Genre_Drama,Genre_Family,Genre_Fantasy,Genre_History,Genre_Horror,Genre_Music,Genre_Mystery,Genre_Romance,Genre_Science Fiction,Genre_TV Movie,Genre_Thriller,Genre_War,Genre_Western
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11816,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
11817,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
11818,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0
11819,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [24]:
Y_pred.sum(axis="columns").value_counts()

2    3555
3    2700
1    2583
4    1409
0     877
5     515
6     141
7      38
8       3
dtype: int64

## Evaluation

In [29]:
movies_test = catalog.load("movies_test")

2021-03-16 14:13:30,799 - kedro.io.data_catalog - INFO - Loading data from `movies_test` (CSVDataSet)...


In [30]:
movies_test

,original_title,poster_path,genre_names,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,...,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
id,,,,,,,,,,,,,,,,,,,,,
14813,Mickey's Christmas Carol,/6QH10DqZ0feBW2phvrtoLqT1s0k.jpg,['Family' 'Animation'],0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7177,Fire from Below,/9eecXogqTUMpsDMNZD8cCA5btMG.jpg,['Action' 'Adventure' 'Science Fiction' 'Thril...,1,1,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
11292,Seventeen - Mädchen sind die besseren Jungs,/oUgbUWfPlAQhKLPnX6N9gAjKkvJ.jpg,['Comedy'],0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19195,Die Pyramide des Sonnengottes,/tFpTyjKRxdm1KEDMhhZd2KDacm1.jpg,['Adventure' 'Western'],0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3074,The Ghost of Frankenstein,/7NtCuLtymDBCWBDj5MVxBjzDUPe.jpg,['Drama' 'Fantasy' 'Horror' 'Science Fiction'],0,0,0,0,0,0,1,...,0,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15470,內衣少女,/1J7IreCxYY7q2e0Yy12yyvbEmWB.jpg,['Comedy' 'Drama' 'Romance'],0,0,0,1,0,0,1,...,0,0,0,0,1,0,0,0,0,0
16218,Le big-Bang,/o4jlPcPQbcYIQ3PLyiCOfP6o2Jd.jpg,['Animation' 'Comedy' 'Science Fiction'],0,0,1,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2637,The Mothman Prophecies,/z2PiOTyjbCOnMHY2QJdq3I4jU44.jpg,['Drama' 'Horror' 'Mystery'],0,0,0,0,0,0,1,...,0,1,0,1,0,0,0,0,0,0


In [32]:
genres = movies_test[movies_test.columns[3:]]
genres

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
id,,,,,,,,,,,,,,,,,,,
14813,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
7177,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
11292,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
19195,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3074,0,0,0,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15470,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
16218,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2637,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0


In [33]:
Y_test = genres

In [34]:
X_test = movies_test[["original_title"]]

In [45]:
scoring = make_scorer(f1_score, greater_is_better=True, average="weighted")
cv = 5

In [46]:
cross_val_score(
    DummyClassifier(strategy="uniform"),
    X_test,
    Y_test,
    scoring=scoring,
    cv=cv,
).mean()

0.27461291972812035

In [47]:
cross_val_score(
    DummyClassifier(strategy="stratified"),
    X_test,
    Y_test,
    scoring=scoring,
    cv=cv,
).mean()

0.22518516056866317

In [48]:
cross_val_score(
    DummyClassifier(strategy="most_frequent"),
    X_test,
    Y_test,
    scoring=scoring,
    cv=cv,
).mean()

0.0